In [3]:
import numpy as np
import os
import pickle 

# Expected shape is 1175 Trials x 13 channels x 2400 samples (@ 5000Hz) - value is potential
raw_source_dir = "NO_FFT_NOT_FLAT/"

files = ['x_train', 'x_test']
# files = ['x_train.pickle', 'y_train.pickle', 'x_test.pickle', 'y_test.pickle']
file_data = []
for i, fn in enumerate(files):
    files[i] = os.path.join(raw_source_dir, fn)
    with open(os.path.join(raw_source_dir, '{}.pickle'.format(fn)), 'rb') as f:
        file_data.append(np.array(pickle.load(f)))
        
# x_train = file_data[0]
# y_train = file_data[1]
# x_test = file_data[2] 
# y_test = file_data[3]
# print(x_train.shape)
# print(y_train.shape)

(1175, 13, 2400)
(1175,)


In [7]:
# Referencing http://forrestbao.blogspot.com/2009/10/eeg-signal-processing-in-python-and.html
# And https://www.ritchievink.com/blog/2017/04/23/understanding-the-fourier-transform-by-example/
for i, eeg_data in enumerate(file_data):
     # Shape trials x channels x samples
    fs = 5000.0

    # Create windows
    sample_length = x_train.shape[-1]
    
    # Hyperparams
    window_length = 625
    step_size = 25
    banded_freq = 200 # Number of freq steps to keep -> goes up to window_length (for some reason)
    
    roll_cap = (sample_length - window_length)//step_size + 1 # Number of windows
    roll_ctr = 0

    # Prepare fft
    # f = np.linspace(0,fs, window_length) # 5000/625 -> resolution of about 8Hz...

    fft_over_time = np.zeros((x_train.shape[0], x_train.shape[1], roll_cap, banded_freq))

    # Could consider vectorizing...
    while roll_ctr < roll_cap:
        window_sample = raw_eeg[...,roll_ctr * step_size: roll_ctr * step_size + window_length]
        fft_over_time[..., roll_ctr,:] = np.abs(np.fft.rfft(window_sample)[...,:banded_freq]) / window_length # fft over whole time frame
        roll_ctr += 1
    pickle_name = "{}_fft.sav".format(files[i])
    pickle.dump(fft_over_time, open(pickle_name, 'wb'))
    
    